In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
c_file = "s3://bdx-emr/ukb49672.csv"
cdata = spark.read.csv(c_file,header='true',inferSchema='true')
v_file = "s3://bdx-emr/lifestyling.csv"
vdata = spark.read.csv(v_file,header='true')

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
6,application_1653011070109_0007,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
print((cdata.count(), len(cdata.columns)))
print((vdata.count(), len(vdata.columns)))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

(502415, 14939)
(461, 3)

In [3]:
import re
allfields=cdata.columns
rnm_fields = [re.sub(r'[-.]', 'x', field) for field in allfields]
#print(rnm_fields)
ncdata=cdata.toDF(*rnm_fields)
allfields=ncdata.columns
lifefields=vdata.select('Column1')
lifefields_ls=lifefields.rdd.flatMap(lambda x: x).collect()
col2type=dict(ncdata.dtypes)
lifefields_set=set(lifefields_ls)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
n=len(allfields)
sel_fields=["eid","31x0x0","21000x0x0","21022x0x0","21001x0x0"]
for i in range(n):
    if allfields[i].split("x")[0] in lifefields_set:
        if col2type[allfields[i]] in ["int","double"]:
            sel_fields.append(allfields[i])
print(len(lifefields_set))
life_data=ncdata.select(sel_fields)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

461

In [5]:
import pyspark.sql.functions as F
null_counts = life_data.select([F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in life_data.columns]).collect()[0].asDict()
m=life_data.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
cut_ratio=0.1
rm_v=0
for key in null_counts:
    if null_counts[key]>cut_ratio*m:
        rm_v+=1
n_v=len(life_data.columns)
print("Number of total variables:"+str(n_v))
print("Number of removed variables:"+str(rm_v))
to_drop = [k for k, v in null_counts.items() if v > cut_ratio*m]
life_data_1 = life_data.drop(*to_drop)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Number of total variables:2389
Number of removed variables:2335

In [7]:
life_data_2=life_data_1.na.drop("any")
m_1=life_data_2.count()
print(m_1)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

384100

In [8]:
from pyspark.ml.feature import VectorAssembler, StandardScaler, PCA
assembler = VectorAssembler(inputCols = life_data_2.columns[5:], outputCol = 'features')
life_data_3 = assembler.transform(life_data_2).select(['eid','31x0x0','21000x0x0','21022x0x0','21001x0x0','features'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
life_data_3.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+---------+---------+---------+--------------------+
|    eid|31x0x0|21000x0x0|21022x0x0|21001x0x0|            features|
+-------+------+---------+---------+---------+--------------------+
|1000015|     0|     1001|       64|  20.9339|[3.0,15.0,0.0,0.0...|
|1000027|     0|     1001|       60|  25.4222|[4.0,-1.0,2.0,1.0...|
+-------+------+---------+---------+---------+--------------------+
only showing top 2 rows

In [10]:
scaler = StandardScaler(
    inputCol = 'features', 
    outputCol = 'scaledFeatures',
    withMean = True,
    withStd = True
).fit(life_data_3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
life_data_4 = scaler.transform(life_data_3)
life_data_4.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+---------+---------+---------+--------------------+--------------------+
|    eid|31x0x0|21000x0x0|21022x0x0|21001x0x0|            features|      scaledFeatures|
+-------+------+---------+---------+---------+--------------------+--------------------+
|1000015|     0|     1001|       64|  20.9339|[3.0,15.0,0.0,0.0...|[-1.3996748400160...|
|1000027|     0|     1001|       60|  25.4222|[4.0,-1.0,2.0,1.0...|[-0.8378360547386...|
+-------+------+---------+---------+---------+--------------------+--------------------+
only showing top 2 rows

In [12]:
n_components = 3
pca = PCA(
    k = n_components, 
    inputCol = 'scaledFeatures', 
    outputCol = 'pcaFeatures'
).fit(life_data_4)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
life_data_5 = pca.transform(life_data_4)
print('Explained Variance Ratio', pca.explainedVariance.toArray())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Explained Variance Ratio [0.06018083 0.05220873 0.04698737]

In [14]:
life_data_5.show(2)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+---------+---------+---------+--------------------+--------------------+--------------------+
|    eid|31x0x0|21000x0x0|21022x0x0|21001x0x0|            features|      scaledFeatures|         pcaFeatures|
+-------+------+---------+---------+---------+--------------------+--------------------+--------------------+
|1000015|     0|     1001|       64|  20.9339|[3.0,15.0,0.0,0.0...|[-1.3996748400160...|[1.34067844331761...|
|1000027|     0|     1001|       60|  25.4222|[4.0,-1.0,2.0,1.0...|[-0.8378360547386...|[-1.4234630630256...|
+-------+------+---------+---------+---------+--------------------+--------------------+--------------------+
only showing top 2 rows

In [15]:
from pyspark.ml.regression import LinearRegression
lr=LinearRegression(featuresCol='pcaFeatures', labelCol='21001x0x0',regParam=0.3, elasticNetParam=0.8)
lr_model = lr.fit(life_data_5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))
print("MSE: ", lr_model.summary.meanSquaredError)
print("MAE: ", lr_model.summary.meanAbsoluteError)
print("R-squared: ", lr_model.summary.r2)
#print("P-value: ", lr_model.summary.pValues)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Coefficients: [0.2337845565472844,-0.31749966756049514,0.0]
Intercept: 27.346174489976566
MSE:  20.56061500086945
MAE:  3.4489244397392107
R-squared:  0.03984014677669134